<a href="https://colab.research.google.com/github/MengOonLee/Google_cloud/blob/main/Infrastructure/BasicsCloudCompute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create a Virtual Machine

### Task 1. Create a new instance from the Cloud console

In [ ]:
gcloud auth list

gcloud config list project

gcloud config set compute/region REGION

export REGION=REGION

export ZONE=Zone

### Task 2. Install an NGINX web server

In [ ]:
sudo apt-get update

sudo apt-get install -y nginx

ps auwx | grep nginx

### Task 3. Create a new instance with gcloud

In [ ]:
gcloud compute instances create gcelab2 --machine-type e2-medium --zone=$ZONE

gcloud compute instances create --help

gcloud compute ssh gcelab2 --zone=Zone


## Creating a Persistent Disk

In [ ]:
gcloud auth list

gcloud config list project

gcloud config set compute/zone Zone
gcloud config set compute/region Region

export REGION=Region

export ZONE=Zone

### Task 1. Create a new instance

In [ ]:
gcloud compute instances create gcelab --zone $ZONE --machine-type e2-standard-2

### Task 2. Create a new persistent disk

In [ ]:
gcloud compute disks create mydisk --size=200GB \
--zone $ZONE

### Task 3. Attaching a disk

In [ ]:
gcloud compute instances attach-disk gcelab --disk mydisk --zone $ZONE


gcloud compute ssh gcelab --zone $ZONE


ls -l /dev/disk/by-id/

# Formatting and mounting the persistent disk

sudo mkdir /mnt/mydisk

sudo mkfs.ext4 -F -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1

sudo mount -o discard,defaults /dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1 /mnt/mydisk

# Automatically mount the disk on restart

sudo nano /etc/fstab

/dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1 /mnt/mydisk ext4 defaults 1 1

## Host a Web App on Google Cloud Using Compute Engine

In [ ]:
gcloud auth list

gcloud config list project

gcloud config set compute/zone "ZONE"
export ZONE=$(gcloud config get compute/zone)

gcloud config set compute/region "REGION"
export REGION=$(gcloud config get compute/region)

### Task 1. Enable the Compute Engine API

In [ ]:
gcloud services enable compute.googleapis.com

### Task 2. Create a Cloud Storage bucket

In [ ]:
gsutil mb gs://fancy-store-Project ID

### Task 3. Clone a source repository

In [ ]:
git clone https://github.com/googlecodelabs/monolith-to-microservices.git

cd ~/monolith-to-microservices

./setup.sh

nvm install --lts

cd microservices
npm start

gcloud services enable cloudaicompanion.googleapis.com

### Task 4. Create the Compute Engine instances

In [ ]:
touch ~/monolith-to-microservices/startup-script.sh

gsutil cp ~/monolith-to-microservices/startup-script.sh gs://fancy-store-Project ID

cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-Project ID/

gcloud compute instances create backend \
    --zone=zone \
    --machine-type=e2-standard-2 \
    --tags=backend \
   --metadata=startup-script-url=https://storage.googleapis.com/fancy-store-Project ID/startup-script.sh

gcloud compute instances list

cd ~/monolith-to-microservices/react-app
npm install && npm run-script build

cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-Project ID/

gcloud compute instances create frontend \
    --zone=zone \
    --machine-type=e2-standard-2 \
    --tags=frontend \
    --metadata=startup-script-url=https://storage.googleapis.com/fancy-store-Project ID/startup-script.sh

gcloud compute firewall-rules create fw-fe \
    --allow tcp:8080 \
    --target-tags=frontend

gcloud compute firewall-rules create fw-be \
    --allow tcp:8081-8082 \
    --target-tags=backend

gcloud compute instances list

### Task 5. Create managed instance groups

In [ ]:
gcloud compute instances stop frontend --zone=zone

gcloud compute instances stop backend --zone=zone

gcloud compute instance-templates create fancy-fe \
    --source-instance-zone=zone \
    --source-instance=frontend

gcloud compute instance-templates create fancy-be \
    --source-instance-zone=zone \
    --source-instance=backend

gcloud compute instance-templates list

gcloud compute instances delete backend --zone=zone

gcloud compute instance-groups managed create fancy-fe-mig \
    --zone=zone \
    --base-instance-name fancy-fe \
    --size 2 \
    --template fancy-fe

gcloud compute instance-groups managed create fancy-be-mig \
    --zone=zone \
    --base-instance-name fancy-be \
    --size 2 \
    --template fancy-be

gcloud compute instance-groups set-named-ports fancy-fe-mig \
    --zone=zone \
    --named-ports frontend:8080

gcloud compute instance-groups set-named-ports fancy-be-mig \
    --zone=zone \
    --named-ports orders:8081,products:8082

gcloud compute health-checks create http fancy-fe-hc \
    --port 8080 \
    --check-interval 30s \
    --healthy-threshold 1 \
    --timeout 10s \
    --unhealthy-threshold 3

gcloud compute health-checks create http fancy-be-hc \
    --port 8081 \
    --request-path=/api/orders \
    --check-interval 30s \
    --healthy-threshold 1 \
    --timeout 10s \
    --unhealthy-threshold 3

gcloud compute firewall-rules create allow-health-check \
    --allow tcp:8080-8081 \
    --source-ranges 130.211.0.0/22,35.191.0.0/16 \
    --network default

gcloud compute instance-groups managed update fancy-fe-mig \
    --zone=zone \
    --health-check fancy-fe-hc \
    --initial-delay 300

gcloud compute instance-groups managed update fancy-be-mig \
    --zone=zone \
    --health-check fancy-be-hc \
    --initial-delay 300

### Task 6. Create load balancers

In [ ]:
gcloud compute http-health-checks create fancy-fe-frontend-hc \
  --request-path / \
  --port 8080

gcloud compute http-health-checks create fancy-be-orders-hc \
  --request-path /api/orders \
  --port 8081

gcloud compute http-health-checks create fancy-be-products-hc \
  --request-path /api/products \
  --port 8082

gcloud compute backend-services create fancy-fe-frontend \
  --http-health-checks fancy-fe-frontend-hc \
  --port-name frontend \
  --global

gcloud compute backend-services create fancy-be-orders \
  --http-health-checks fancy-be-orders-hc \
  --port-name orders \
  --global

gcloud compute backend-services create fancy-be-products \
  --http-health-checks fancy-be-products-hc \
  --port-name products \
  --global

gcloud compute backend-services add-backend fancy-fe-frontend \
  --instance-group-zone=zone \
  --instance-group fancy-fe-mig \
  --global

gcloud compute backend-services add-backend fancy-be-orders \
  --instance-group-zone=zone \
  --instance-group fancy-be-mig \
  --global

gcloud compute backend-services add-backend fancy-be-products \
  --instance-group-zone=zone \
  --instance-group fancy-be-mig \
  --global

gcloud compute url-maps create fancy-map \
  --default-service fancy-fe-frontend

gcloud compute url-maps add-path-matcher fancy-map \
   --default-service fancy-fe-frontend \
   --path-matcher-name orders \
   --path-rules "/api/orders=fancy-be-orders,/api/products=fancy-be-products"

gcloud compute target-http-proxies create fancy-proxy \
  --url-map fancy-map

gcloud compute forwarding-rules create fancy-http-rule \
  --global \
  --target-http-proxy fancy-proxy \
  --ports 80

cd ~/monolith-to-microservices/react-app/

gcloud compute forwarding-rules list --global

cd ~/monolith-to-microservices/react-app
npm install && npm run-script build

cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-Project ID/

gcloud compute instance-groups managed rolling-action replace fancy-fe-mig \
    --zone=zone \
    --max-unavailable 100%

watch -n 2 gcloud compute backend-services get-health fancy-fe-frontend --global

### Task 7. Scale Compute Engine

In [ ]:
gcloud compute instance-groups managed set-autoscaling \
  fancy-fe-mig \
  --zone=zone \
  --max-num-replicas 2 \
  --target-load-balancing-utilization 0.60

gcloud compute instance-groups managed set-autoscaling \
  fancy-be-mig \
  --zone=zone \
  --max-num-replicas 2 \
  --target-load-balancing-utilization 0.60

gcloud compute backend-services update fancy-fe-frontend \
    --enable-cdn --global

### Task 8. Update the website

In [ ]:
gcloud compute instances set-machine-type frontend \
  --zone=zone \
  --machine-type e2-small

gcloud compute instance-templates create fancy-fe-new \
    --region=$REGION \
    --source-instance=frontend \
    --source-instance-zone=zone

gcloud compute instance-groups managed rolling-action start-update fancy-fe-mig \
  --zone=zone \
  --version template=fancy-fe-new

watch -n 2 gcloud compute instance-groups managed list-instances fancy-fe-mig \
  --zone=zone

gcloud compute instances describe [VM_NAME] --zone=zone | grep machineType

cd ~/monolith-to-microservices/react-app/src/pages/Home
mv index.js.new index.js

cat ~/monolith-to-microservices/react-app/src/pages/Home/index.js

cd ~/monolith-to-microservices/react-app
npm install && npm run-script build

cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-Project ID/

gcloud compute instance-groups managed rolling-action replace fancy-fe-mig \
  --zone=zone \
  --max-unavailable=100%

watch -n 2 gcloud compute backend-services get-health fancy-fe-frontend --global

gcloud compute forwarding-rules list --global

gcloud compute instance-groups list-instances fancy-fe-mig --zone=zone

gcloud compute ssh [INSTANCE_NAME] --zone=zone

sudo supervisorctl stop nodeapp; sudo killall node

exit

watch -n 2 gcloud compute operations list \
--filter='operationType~compute.instances.repair.*'

## The Basics of Google Cloud Compute: Challenge Lab

### Task 1. Create a Cloud Storage bucket

In [ ]:
gcloud auth list

gcloud config list project

gcloud config set compute/zone Zone
gcloud config set compute/region Region

export REGION=Region

export ZONE=Zone

gsutil mb gs://fancy-store-Project ID

### Task 2. Create and attach a persistent disk to a Compute Engine instance

In [ ]:
gcloud compute instances create gcelab --zone $ZONE --machine-type e2-standard-2

gcloud compute disks create mydisk --size=200GB \
--zone $ZONE

gcloud compute instances attach-disk gcelab --disk mydisk --zone $ZONE

gcloud compute ssh gcelab --zone $ZONE


ls -l /dev/disk/by-id/

# Formatting and mounting the persistent disk

sudo mkdir /mnt/mydisk

sudo mkfs.ext4 -F -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1

sudo mount -o discard,defaults /dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1 /mnt/mydisk

# Automatically mount the disk on restart

sudo nano /etc/fstab

/dev/disk/by-id/scsi-0Google_PersistentDisk_persistent-disk-1 /mnt/mydisk ext4 defaults 1 1

### Task 3. Install a NGINX web server